In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_df = pd.read_csv('/kaggle/input/spaceship-titanic/train.csv')
test_df = pd.read_csv('/kaggle/input/spaceship-titanic/test.csv')
sub_df = pd.read_csv('/kaggle/input/spaceship-titanic/sample_submission.csv')

In [ ]:
train_df.head()

In [ ]:
train_df.isnull().sum()

In [ ]:
test_df.isnull().sum()

In [ ]:
def process_cabin(df):
    df[["Deck","Num","Side"]] = df["Cabin"].str.split("/", expand = True)
    df.drop(columns = ["Cabin"], inplace = True)
    return df

In [ ]:
train_df = process_cabin(train_df)
test_df = process_cabin(test_df)

In [ ]:
bool_cols = ["CryoSleep", "VIP", "Transported"]
for cols in bool_cols:
    train_df[cols] = train_df[cols].astype(float)
    if cols in test_df.columns:
        test_df[cols] = test_df[cols].astype(float)
        

In [ ]:
train_df.head()

In [ ]:
train_df.drop(columns = ["Name"], inplace = True)
test_df.drop(columns = ["Name"], inplace = True)

In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline

categorical_cols = ["HomePlanet", "Destination", "Deck", "Side"]
numerical_cols = ["CryoSleep", "Age", "VIP", "RoomService", "FoodCourt", "ShoppingMall","Spa", "VRDeck","Num"]

num_imputer = SimpleImputer(strategy = "median")
cat_imputer = SimpleImputer(strategy = "most_frequent")
encoder = OneHotEncoder(handle_unknown = "ignore")

preprocessor = ColumnTransformer([
    ("num", num_imputer, numerical_cols),
    ("cat", Pipeline(steps = [("imputer", cat_imputer), ("encoder", encoder)]), categorical_cols)
])

model = RandomForestClassifier(n_estimators = 100, random_state = 42)

pipeline = Pipeline([
    ("preprocessor", preprocessor),
    ("model", model)
])
X = train_df.drop(columns = ["Transported", "PassengerId"])
Y = train_df["Transported"]
X_test = test_df.drop(columns = ["PassengerId"])
pipeline.fit(X,Y)
y_pred = pipeline.predict(X_test)
print ("Model Trained")


In [ ]:
sub_df.head()

In [ ]:
output = pd.DataFrame({'PassengerId': test_df.PassengerId, 'Transported': y_pred})


In [ ]:
output.head()

In [ ]:
output["Transported"] = output["Transported"].astype(bool)
output.to_csv("submission.csv", index = False)
